# Sonnet MNIST Example

In [1]:
#!pip install dm-sonnet

In [2]:
#!pip install tensorflow_probability

Sonnet is an object-oriented library written in Python. It was released by DeepMind in 2017. 

Sonnet intends to cleanly separate the following two aspects of building computation graphs from objects: 

The configuration of objects called modules 
The connection of objects to computation graphs 


The modules are defined as sub-classes of the abstract class sonnet.AbstractModule. 

The following modules are available in Sonnet: 

Basic modules: AddBias, BatchApply, BatchFlatten, BatchReshape, FlattenTrailingDimensions, Linear, MergeDims, SelectInput, SliceByDim, TileByDim, and TrainableVariable 

Recurrent modules: DeepRNN, ModelRNN, VanillaRNN, BatchNormLSTM, GRU, and LSTM 

Recurrent + ConvNet modules: Conv1DLSTM and Conv2DLSTM ConvNet modules Conv1D, Conv2D, Conv3D, Conv1DTranspose, Conv2DTranspose, Conv3DTranspose, DepthWiseConv2D, InPlaneConv2D, and SeparableConv2D 

ResidualNets: Residual, ResidualCore, and SkipConnectionCore 

Others: BatchNorm, LayerNorm, clip_gradient, and scale_gradient



In [3]:
import tensorflow as tf
tf.reset_default_graph()

import os
import sonnet as snt

from tensorflow.examples.tutorials.mnist import input_data

tf.logging.set_verbosity(tf.logging.INFO)


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [4]:
class MNIST(snt.AbstractModule):

    def __init__(self, mnist_part, batch_size, name='MNIST'):

        super(MNIST, self).__init__(name=name)

        self._X = tf.constant(mnist_part.images, dtype=tf.float32)
        self._Y = tf.constant(mnist_part.labels, dtype=tf.float32)
        self._batch_size = batch_size
        self._M = mnist_part.num_examples

    def _build(self):
        idx = tf.random_uniform([self._batch_size], 0, self._M, tf.int64)
        X = tf.gather(self._X, idx)
        Y = tf.gather(self._Y, idx)
        return X, Y


In [5]:
class MLP(snt.AbstractModule):
    def __init__(self, output_sizes, name='mlp'):
        super(MLP, self).__init__(name=name)

        self._layers = []

        for output_size in output_sizes:
            self._layers.append(snt.Linear(output_size=output_size))

    def _build(self, X):

        # add the input layer
        model = tf.sigmoid(self._layers[0](X))

        # add hidden layers
        for i in range(1, len(self._layers) - 1):
            model = tf.sigmoid(self._layers[i](model))

        # add output layer
        model = tf.nn.softmax(self._layers[len(self._layers) - 1](model))

        return model


In [6]:
batch_size = 100
n_classes = 10
n_epochs = 10


In [7]:
mnist = input_data.read_data_sets(os.path.join('.', 'mnist'),
                                  one_hot=True
                                  )

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
train = MNIST(mnist.train, batch_size=batch_size)
test = MNIST(mnist.test, batch_size=batch_size)

X_train, Y_train = train()
X_test, Y_test = test()


In [9]:
model = MLP([20, n_classes])

In [10]:
Y_train_hat = model(X_train)
Y_test_hat = model(X_test)


In [11]:
def loss(Y_hat, Y):
    return -tf.reduce_sum(Y * tf.log(Y_hat))



In [12]:
L_train = loss(Y_train_hat, Y_train)
L_test = loss(Y_test_hat, Y_test)


In [13]:
optimizer = tf.train.GradientDescentOptimizer(
    learning_rate=0.01).minimize(L_train)


In [14]:

with tf.Session() as tfs:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        loss_val, _ = tfs.run((L_train, optimizer))
        print('Epoch : {} Training Loss : {}'.format(epoch, loss_val))

    loss_val = tfs.run(L_test)
    print('Test loss : {}'.format(loss_val))

Epoch : 0 Training Loss : 232.91702270507812
Epoch : 1 Training Loss : 228.2225341796875
Epoch : 2 Training Loss : 226.57925415039062
Epoch : 3 Training Loss : 222.71978759765625
Epoch : 4 Training Loss : 211.66622924804688
Epoch : 5 Training Loss : 207.2665252685547
Epoch : 6 Training Loss : 214.3721160888672
Epoch : 7 Training Loss : 200.88914489746094
Epoch : 8 Training Loss : 194.6040496826172
Epoch : 9 Training Loss : 189.76661682128906
Test loss : 181.6356201171875
